### Training of modified U-Net for Optic Disc on RIM-ONE v3 database (cross-validation fold #0).

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt

from model_rimone import *
from process_images_rimone import *
from process_result import *

print('Keras version:', keras.__version__)
print('TensorFlow version:', tf.__version__)

In [ ]:
images, cups = get_images()

In [ ]:
gen = data_generator(images, cups, 128, 'train', batch_size=1)
batch = next(gen)
batch[0].shape

In [ ]:
fig = plt.imshow(batch[0][0])
plt.show()
plt.imshow(batch[1][0]); plt.show()

In [ ]:
X_valid, Y_valid = next(data_generator(images, cups, train_or_test='test', batch_size=100, stationary=True))
plt.imshow(X_valid[20]); plt.show()
plt.imshow(Y_valid[20]); plt.show()
print(X_valid.shape, Y_valid.shape)

In [ ]:
epochs = NUM_EPOCHS
spe = SPE
img_size = IMG_SIZE
arq_name = 'lr_7_e_4_gap_70'

model = create_compile_model(img_size, 3e-4)
history_cup = train_cup(images, 
                     cups, 
                     'cup_color_rimone', 
                     model, epochs, X_valid, Y_valid, img_size, spe)

In [ ]:
pred_iou, pred_dice, pred_cup = predict(images, X_valid, Y_valid, model, img_size)

In [ ]:
folder_name = root_path(arq_name)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, 'rgb_color_mean_iou_dice_cup.csv')

In [ ]:
images, discs = get_images_disc()

X_valid, Z_valid = next(data_generator(images, discs, train_or_test='test', batch_size=100, stationary=True))
plt.imshow(X_valid[20]); plt.show()
plt.imshow(Z_valid[20]); plt.show()
print(X_valid.shape, Z_valid.shape)

In [ ]:
model_disc = create_compile_model(img_size, 1e-3)
history_disc = train_disc(images, 
                     discs, 
                     'disc_color_rimone', 
                     model_disc, epochs, X_valid, Z_valid, img_size, spe)

In [ ]:
pred_iou, pred_dice, pred_disc = predict(images, X_valid, Z_valid, model_disc, img_size)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, 'rgb_color_mean_iou_dice_disc.csv')

In [ ]:
plot_results(history_cup, epochs, folder_name, arq_name+'_cup')

In [ ]:
plot_results(history_disc, epochs, folder_name, arq_name+'_disc')

In [ ]:
df, diametros = create_table_result_rimone(pred_cup, pred_disc, test_idx, folder_name, 'rgb_color') 

In [ ]:
print(df)

In [ ]:
print(diametros)